In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models
from medmnist import INFO, Evaluator
from medmnist.dataset import PneumoniaMNIST
from torch.utils.data import random_split

## Hyperparameters

In [3]:
batch_size = 64
learning_rate = 0.001
num_epochs = 30

## Datasets and Dataloaders

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

full_train_dataset = PneumoniaMNIST(split='train', transform=transform, download=True)

train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

test_dataset = PneumoniaMNIST(split='test', transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Using downloaded and verified file: C:\Users\tanji\.medmnist\pneumoniamnist.npz
Using downloaded and verified file: C:\Users\tanji\.medmnist\pneumoniamnist.npz


## Accuracy Calculation Function

In [5]:
def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device).squeeze().long()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # print(f"Predicted: {predicted[:5]}")
            # print(f"Labels: {labels[:5]}")
            # print(f"Batch Correct: {(predicted == labels).sum().item()}")
            # print(f"Batch Total: {labels.size(0)}")
    accuracy = (correct / total) * 100
    return accuracy

## ResNet18 Model

In [6]:
class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

num_classes = len(INFO['pneumoniamnist']['label'])
model = ResNet18(num_classes=num_classes)

## Loss Function and Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Train

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device).squeeze().long()

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_accuracy = calculate_accuracy(train_loader, model)
        val_accuracy = calculate_accuracy(val_loader, model)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, '
              f'Train Accuracy: {train_accuracy:.2f}%, Val Accuracy: {val_accuracy:.2f}%')

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)

Epoch [1/30], Loss: 0.2449, Train Accuracy: 95.43%, Val Accuracy: 94.59%
Epoch [2/30], Loss: 0.1398, Train Accuracy: 97.27%, Val Accuracy: 96.60%
Epoch [3/30], Loss: 0.0867, Train Accuracy: 93.10%, Val Accuracy: 93.21%
Epoch [4/30], Loss: 0.0871, Train Accuracy: 95.57%, Val Accuracy: 94.27%
Epoch [5/30], Loss: 0.0736, Train Accuracy: 97.29%, Val Accuracy: 96.18%
Epoch [6/30], Loss: 0.0597, Train Accuracy: 96.31%, Val Accuracy: 94.69%
Epoch [7/30], Loss: 0.0528, Train Accuracy: 97.34%, Val Accuracy: 95.33%
Epoch [8/30], Loss: 0.0545, Train Accuracy: 97.24%, Val Accuracy: 95.97%
Epoch [9/30], Loss: 0.0743, Train Accuracy: 97.72%, Val Accuracy: 96.07%
Epoch [10/30], Loss: 0.0496, Train Accuracy: 97.16%, Val Accuracy: 93.84%
Epoch [11/30], Loss: 0.0415, Train Accuracy: 98.06%, Val Accuracy: 95.22%
Epoch [12/30], Loss: 0.0350, Train Accuracy: 99.39%, Val Accuracy: 97.13%
Epoch [13/30], Loss: 0.0317, Train Accuracy: 95.83%, Val Accuracy: 94.37%
Epoch [14/30], Loss: 0.0306, Train Accuracy: 98

## Evaluate

In [9]:
def evaluate_model(model, test_loader):
    test_accuracy = calculate_accuracy(test_loader, model)
    print(f'Accuracy of the model on the test images: {test_accuracy:.2f}%')

evaluate_model(model, test_loader)

Accuracy of the model on the test images: 85.90%


## Train

In [10]:
torch.save(model.state_dict(), 'resnet18_pneumonia_mnist.pth')